In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/mukulvarshney09/Diamon_Price_Prediction/main/notebooks/data/gemstone.csv")

In [3]:
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [4]:
df.drop(labels = ["id"], axis = 1, inplace = True)

In [5]:
df.head()

,carat,cut,color,clarity,depth,table,x,y,z,price
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [6]:
x = df.drop(labels = ["price"], axis = 1)
y = df[["price"]]

In [7]:
x

,carat,cut,color,clarity,depth,table,x,y,z
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77
...,...,...,...,...,...,...,...,...,...
193568,0.31,Ideal,D,VVS2,61.1,56.0,4.35,4.39,2.67
193569,0.70,Premium,G,VVS2,60.3,58.0,5.75,5.77,3.47
193570,0.73,Very Good,F,SI1,63.1,57.0,5.72,5.75,3.62
193571,0.34,Very Good,D,SI1,62.9,55.0,4.45,4.49,2.81


In [8]:
y

,price
0,13619
1,13387
2,2772
3,666
4,14453
...,...
193568,1130
193569,2874
193570,3036
193571,681


In [9]:
cat_cols = x.select_dtypes(include="O").columns
num_cols = x.select_dtypes(exclude="O").columns

In [10]:
cat_cols

Index(['cut', 'color', 'clarity'], dtype='object')

In [11]:
num_cols

Index(['carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object')

In [12]:
df["cut"].unique()

array(['Premium', 'Very Good', 'Ideal', 'Good', 'Fair'], dtype=object)

In [13]:
cut_categories = ["Fair", "Good", "Very Good", "Premium", "Ideal"]

In [14]:
df["color"].unique()

array(['F', 'J', 'G', 'E', 'D', 'H', 'I'], dtype=object)

In [15]:
color_categories = ["D","E","F","G","H","I","J"]

In [16]:
df["clarity"].unique()

array(['VS2', 'SI2', 'VS1', 'SI1', 'IF', 'VVS2', 'VVS1', 'I1'],
      dtype=object)

In [17]:
clarity_categories = ["I1", "SI2", "SI1", "VS2", "VS1", "VVS2", "VVS1","IF"]

In [18]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.pipeline import Pipeline

from sklearn.compose import ColumnTransformer

In [19]:
#now need to create pipelines

In [20]:
cat_pipeline = Pipeline(steps=[("imputer",SimpleImputer(strategy="most_frequent")),
                       ("encoder",OrdinalEncoder(categories=[cut_categories, color_categories, clarity_categories])),
                       ("scaler",StandardScaler()) ]    )

In [21]:
num_pipeline = Pipeline(
                steps = [("imputer", SimpleImputer(strategy="median")),
                         ("scaler", StandardScaler())               
                ])

In [22]:
preprocessor = ColumnTransformer([
                ("numerical",num_pipeline,num_cols),
                ("categorical",cat_pipeline, cat_cols)
                ])

In [23]:
#now the pipeline structure is done, need to divide the data into train test and the fit into the pipeline function

In [24]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x,y,random_state=69,test_size=0.20)

In [25]:
preprocessor.fit_transform(x_train)

array([[-0.17508031, -1.0349248 ,  1.44342123, ..., -0.13269012,
        -1.55017746, -0.64950981],
       [ 1.81329567, -0.11094681,  0.92223338, ..., -1.13926045,
        -0.31736907,  0.01614952],
       [-0.54249762,  0.07384879, -0.12014232, ...,  0.87388021,
         0.29903513,  0.68180885],
       ...,
       [-1.06120439,  0.25864438, -0.64133017, ...,  0.87388021,
         0.91543932,  1.34746818],
       [ 1.55394228,  0.81303118, -0.64133017, ..., -0.13269012,
         0.91543932,  0.01614952],
       [ 0.49491594, -0.38814021, -0.12014232, ...,  0.87388021,
        -0.31736907,  0.01614952]])

In [26]:
x_train = pd.DataFrame(preprocessor.fit_transform(x_train), columns=preprocessor.get_feature_names_out())
x_test = pd.DataFrame(preprocessor.transform(x_test), columns=preprocessor.get_feature_names_out())

In [27]:
#now data is modified, need to import the model and required parameters to check accuracy

In [28]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, accuracy_score

In [29]:
lin_reg = LinearRegression()    

In [30]:
lin_reg.fit(x_train, y_train) 

LinearRegression()

In [31]:
y_pred = lin_reg.predict(x_test)  

In [32]:
score = r2_score(y_test, y_pred)    
score

0.9340166937428287

In [33]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, accuracy_score

lin_reg = LinearRegression()
lin_reg.fit(x_train, y_train)
y_pred = lin_reg.predict(x_test)
score = r2_score(y_test, y_pred)
score

0.9340166937428287

In [34]:
lasso_rig = Lasso()

In [35]:
lasso_rig.fit(x_train, y_train)
y_pred = lasso_rig.predict(x_test)
score2 = r2_score(y_test, y_pred)
score2*100

93.4046829905509

In [36]:
ridge_rig = Ridge()
ridge_rig.fit(x_train, y_train)
y_pred = ridge_rig.predict(x_test)
score3 = r2_score(y_test, y_pred)
score3*100

93.40170690836797

In [ ]:
#now let's automate this along with checking mae, mse and rmse